In [3]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# 2. Draw Hands
<img src=https://i.imgur.com/qpRACer.png />

In [5]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))

        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    print(image.shape)

    cap.release()
    cv2.destroyAllWindows()

(480, 640, 3)


In [15]:
results.multi_hand_landmarks

[landmark {
   x: 0.1204281598329544
   y: 0.680406391620636
   z: 0.0
 }
 landmark {
   x: 0.22263863682746887
   y: 0.6174689531326294
   z: -0.04750192165374756
 }
 landmark {
   x: 0.3030247986316681
   y: 0.5117108225822449
   z: -0.06817018985748291
 }
 landmark {
   x: 0.35664963722229004
   y: 0.4073064625263214
   z: -0.08916173130273819
 }
 landmark {
   x: 0.401302695274353
   y: 0.3241441249847412
   z: -0.11012090742588043
 }
 landmark {
   x: 0.20998145639896393
   y: 0.2994726896286011
   z: -0.008263284340500832
 }
 landmark {
   x: 0.2416376918554306
   y: 0.1559467911720276
   z: -0.0366806834936142
 }
 landmark {
   x: 0.26589083671569824
   y: 0.06693273782730103
   z: -0.060824859887361526
 }
 landmark {
   x: 0.29197004437446594
   y: -0.00497359037399292
   z: -0.0774669200181961
 }
 landmark {
   x: 0.13469147682189941
   y: 0.27844271063804626
   z: -0.014230594970285892
 }
 landmark {
   x: 0.13003697991371155
   y: 0.11188454926013947
   z: -0.034049682319164

# Detect Left and Right Hand

In [20]:
results.multi_hand_landmarks[1].landmark[mp_hands.HandLandmark.WRIST]

x: 0.7342785000801086
y: 0.7525222301483154
z: 0.0

In [17]:
results.multi_handedness[0].classification[0]

index: 0
score: 0.976839005947113
label: "Left"

In [3]:
def get_label(index, hand, results):
    output=None
    for idx, classification in enumerate(results.multi_handedness):
        if classification.classification[0].index == index:
            
            label = classification.classification[0].label
            score = classification.classification[0].score
            text = '{} {}'.format(label, round(score, 2))
            
            coords = tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.THUMB_TIP].x, hand.landmark[mp_hands.HandLandmark.THUMB_TIP].y)),
                         [640, 480]).astype(int))
            
            output = text, coords
        
    return output

In [11]:
get_label(num, hand, results)

('Left 0.97', (210, 121))

In [18]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #print(results)
        
        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
                if get_label(num, hand, results):
                    text, coord = get_label(num, hand, results)
                    cv2.putText(image, text, coord, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Line Between Fingers

In [23]:
x_index_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x
y_index_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y

x_thumb_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].x
y_thumb_finger = results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].y

0.4825209379196167

In [4]:
def get_position(landmark_idx = mp_hands.HandLandmark.INDEX_FINGER_TIP):
    x = int(results.multi_hand_landmarks[0].landmark[landmark_idx].x * 640)
    y = int(results.multi_hand_landmarks[0].landmark[landmark_idx].y * 480)
    z = results.multi_hand_landmarks[0].landmark[landmark_idx].y
    
    return x, y, z

def get_distance(x1, y1, x2, y2):
    distance = ((x1-x2)**2 + (y1-y2)**2)**0.5
    return distance
    

In [36]:
np.array([3, 3])

array([3, 3])

In [5]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Flip horizontal
        image = cv2.flip(image, 1)
        
        #Set flag
        image.flags.writeable = False
        
        #Detections
        results = hands.process(image)
        
        #Set flag back to True
        image.flags.writeable = True
        
        #RGB to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        #Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                         mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4), 
                                         mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=4))
                
                
            x1, y1, z1 = get_position(mp_hands.HandLandmark.INDEX_FINGER_MCP)
            x2, y2, z2 = get_position(mp_hands.HandLandmark.WRIST)
            base = int(get_distance(x1, y1, x2, y2))
                
            x1, y1, z1 = get_position(mp_hands.HandLandmark.INDEX_FINGER_TIP)
            #cv2.putText(image, 'INDEX', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            x2, y2, z2 = get_position(mp_hands.HandLandmark.THUMB_TIP)
            #cv2.putText(image, 'THUMB', (x2, y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            cv2.line(image, (x1, y1), (x2, y2), (255, 125, 100), 2)
            
            dist = str(int(100*get_distance(x1, y1, x2, y2)/base)) 
            cv2.putText(image, dist, (x2, y2), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            #cv2.putText(image, str(z1), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
        #image = cv2.flip(image, 0)
        cv2.imshow("Hand Tracking", image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()